<a href="https://colab.research.google.com/github/haruhiysd/QM-Group-Project/blob/main/ANOVA_for_seasonal_differences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install pingouin
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.2 MB/s eta 0:00:00


In [38]:
df=pd.read_csv('/merged df 1.csv')

In [39]:
rename_dict = {
    '01 Andaluc√≠a': 'Andalucía',
    '02 Arag√≥n': 'Aragón',
    '03 Asturias, Principado de': 'Principado de Asturias',
    '04 Balears, Illes': 'Illes Balears',
    '05 Canarias': 'Canarias',
    '06 Cantabria': 'Cantabria',
    '07 Castilla y Le√≥n': 'Castilla y León',
    '08 Castilla - La Mancha': 'Castilla-La Mancha',
    '09 Catalu√±a': 'Cataluña',
    '10 Comunitat Valenciana': 'Comunitat Valenciana',
    '11 Extremadura': 'Extremadura',
    '12 Galicia': 'Galicia',
    '13 Madrid, Comunidad de': 'Comunidad de Madrid',
    '14 Murcia, Regi√≥n de': 'Región de Murcia',
    '15 Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra',
    '16 Pa√≠s Vasco': 'País Vasco',
    '17 Rioja, La': 'La Rioja',
}

In [40]:
df['Autonomous_community'] = (
    df['Autonomous_community']
    .replace(rename_dict)
)

In [41]:
df['Period'] = pd.to_datetime(df['Period'], dayfirst=True)

In [42]:
def month_to_quarter(m):
    if m == 1:
        return 'Jan'
    elif m == 4:
        return 'Apr'
    elif m == 7:
        return 'Jul'
    elif m == 10:
        return 'Oct'
    else:
        return 'Other'

df['quarter'] = df['Period'].dt.month.apply(month_to_quarter)

In [43]:
df = df[['Autonomous_community', 'quarter', 'Unemployment_rate', 'HPI']]

In [44]:
df = df.dropna(subset=["Autonomous_community"])
df = df[df["quarter"].isin(["Jan", "Apr", "Jul", "Oct"])]

In [45]:
df = df.sort_values(by=['Autonomous_community', 'quarter'], ascending=[True, True])
df

,Autonomous_community,quarter,Unemployment_rate,HPI
17,Andalucía,Apr,23.09,110.70
0,Andalucía,Jan,24.74,108.28
34,Andalucía,Jul,22.85,112.67
51,Andalucía,Oct,21.26,113.83
18,Aragón,Apr,9.97,108.95
...,...,...,...,...
53,Principado de Asturias,Oct,12.86,107.60
30,Región de Murcia,Apr,16.29,105.16
13,Región de Murcia,Jan,18.60,104.08
47,Región de Murcia,Jul,16.33,107.79


In [51]:
rm_unemp = pg.rm_anova(
    dv='Unemployment_rate',
    within='quarter',
    subject='Autonomous_community',
    data=df,
    detailed=True
)
print("=== Repeated-Measures ANOVA for Unemployment Rate ===")
print(rm_unemp)


posthoc_unemp = pg.pairwise_ttests(
    dv='Unemployment_rate',
    within='quarter',
    subject='Autonomous_community',
    data=df,
    padjust='bonf'
)
print("\n=== Post-hoc comparisons (Unemployment Rate) ===")
print(posthoc_unemp)


rm_hpi = pg.rm_anova(
    dv='HPI',
    within='quarter',
    subject='Autonomous_community',
    data=df,
    detailed=True
)
print("\n=== Repeated-Measures ANOVA for HPI ===")
print(rm_hpi)

posthoc_hpi = pg.pairwise_ttests(
    dv='HPI',
    within='quarter',
    subject='Autonomous_community',
    data=df,
    padjust='bonf'
)
print("\n=== Post-hoc comparisons (HPI) ===")
print(posthoc_hpi)


=== Repeated-Measures ANOVA for Unemployment Rate ===
    Source         SS  DF         MS         F         p-unc  p-GG-corr  \
0  quarter  69.803463   3  23.267821  21.31997  6.402448e-09    0.00001   
1    Error  52.385412  48   1.091363       NaN           NaN        NaN   

        ng2       eps sphericity   W-spher   p-spher  
0  0.049379  0.542922      False  0.241731  0.000871  
1       NaN       NaN        NaN       NaN       NaN  

=== Post-hoc comparisons (Unemployment Rate) ===
  Contrast    A    B  Paired  Parametric         T   dof alternative  \
0  quarter  Apr  Jan    True        True -4.939270  16.0   two-sided   
1  quarter  Apr  Jul    True        True  3.589840  16.0   two-sided   
2  quarter  Apr  Oct    True        True  3.048160  16.0   two-sided   
3  quarter  Jan  Jul    True        True  4.884468  16.0   two-sided   
4  quarter  Jan  Oct    True        True  5.886348  16.0   two-sided   
5  quarter  Jul  Oct    True        True -0.994465  16.0   two-sided   



/usr/local/lib/python3.11/dist-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
/usr/local/lib/python3.11/dist-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

regions = df["Autonomous_community"].unique()

for reg in regions:
    df_sub = df[df["Autonomous_community"] == reg]

    # Check if there are enough observations for each quarter
    if len(df_sub['quarter'].unique()) < 2:  # Need at least 2 quarters for ANOVA
        print(f"\n=== Skipping ANOVA for {reg} due to insufficient data ===")
        continue  # Skip to the next region

    try:  # Added try-except block to handle potential errors
        model_unemp = smf.ols("Unemployment_rate ~ C(quarter)", data=df_sub).fit()
        anova_unemp = sm.stats.anova_lm(model_unemp, typ=2)
        print(f"\n=== ANOVA for Unemployment Rate in {reg} ===")
        print(anova_unemp)
    except ValueError:
        print(f"\n=== ANOVA for Unemployment Rate in {reg} failed due to multicollinearity or insufficient data ===")

    try: # Added try-except block to handle potential errors
        model_hpi = smf.ols("HPI ~ C(quarter)", data=df_sub).fit()
        anova_hpi = sm.stats.anova_lm(model_hpi, typ=2)
        print(f"\n=== ANOVA for HPI in {reg} ===")
        print(anova_hpi)
    except ValueError:
        print(f"\n=== ANOVA for HPI in {reg} failed due to multicollinearity or insufficient data ===")